In [1]:
import os
import sys
from pathlib import Path
from tempfile import TemporaryDirectory
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

C:\Users\rhasan1\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# get current working directory
currentDir = os.getcwd()

In [3]:
# define the sub-model/region
rgn = int(4)
regionName = f'Region{rgn}'

# save the package
toSavePath = os.path.join(currentDir,"new_packahes_path")

In [13]:
# Get the groundwater pumping dataset
# the dataset also has the land surface elevation for each x,y location
dfGwData = pd.read_csv(f'GwUseData_R{rgn}_DEM.csv')

# read the previously saved dataframe that has basic model informations
dfRH = pd.read_csv(f'Region{rgn}RH_basic.csv')

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\3697416159.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfGwData = pd.read_csv(f'GwUseData_R{rgn}_DEM.csv')


In [14]:
RH_basic = dfRH[['cellid', 'cellTop', 'cellBot', 'IC', 'K', 'K33',
       'SS', 'SY', 'X', 'Y', 'IND']]

RH_basic['cellid'] = RH_basic['cellid'].astype(int)
RH_basic['X'] = RH_basic['X'].astype(int)
RH_basic['Y'] = RH_basic['Y'].astype(int)
RH_basic['IND'] = RH_basic['IND'].astype(int)

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\2163869578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RH_basic['cellid'] = RH_basic['cellid'].astype(int)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\2163869578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RH_basic['X'] = RH_basic['X'].astype(int)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\2163869578.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [15]:
# get the middle point of each 3D cell
RH_basic['MidPoint'] = (RH_basic['cellTop']+RH_basic['cellBot'])/2

In [16]:
#dfGwData.columns.values

In [17]:
GwData_all = dfGwData[['Well_Name', 'X_Albers', 'Y_Albers', 'SOURCE_CD', 'PRIN_AQUIF', 'STATE_CD', 'COUNTY_CD',
       'USE_CD', 'State', 'County', 'Aquifer_Co', 'AquiferTop',
       'AquiferBot', 'Well_DIAM', 'WELL_DEPTH', 'GCP_DEM','Top_Scr_1', 'Bot_Scr_1',
       'Top_Scr_2', 'Bot_Scr_2', 'Top_Scr_3', 'Bot_Scr_3', 'Top_Scr_4',
       'Bot_Scr_4', 'Top_Scr_5', 'Bot_Scr_5', 'Top_Scr_6', 'Bot_Scr_6',
       'JAN_1985', 'FEB_1985', 'MAR_1985', 'APR_1985', 'MAY_1985',
       'JUN_1985', 'JUL_1985', 'AUG_1985', 'SEP_1985', 'OCT_1985',
       'NOV_1985', 'DEC_1985', 'JAN_1986', 'FEB_1986', 'MAR_1986',
       'APR_1986', 'MAY_1986', 'JUN_1986', 'JUL_1986', 'AUG_1986',
       'SEP_1986', 'OCT_1986', 'NOV_1986', 'DEC_1986', 'JAN_1987',
       'FEB_1987', 'MAR_1987', 'APR_1987', 'MAY_1987', 'JUN_1987',
       'JUL_1987', 'AUG_1987', 'SEP_1987', 'OCT_1987', 'NOV_1987',
       'DEC_1987', 'JAN_1988', 'FEB_1988', 'MAR_1988', 'APR_1988',
       'MAY_1988', 'JUN_1988', 'JUL_1988', 'AUG_1988', 'SEP_1988',
       'OCT_1988', 'NOV_1988', 'DEC_1988', 'JAN_1989', 'FEB_1989',
       'MAR_1989', 'APR_1989', 'MAY_1989', 'JUN_1989', 'JUL_1989',
       'AUG_1989', 'SEP_1989', 'OCT_1989', 'NOV_1989', 'DEC_1989',
       'JAN_1990', 'FEB_1990', 'MAR_1990', 'APR_1990', 'MAY_1990',
       'JUN_1990', 'JUL_1990', 'AUG_1990', 'SEP_1990', 'OCT_1990',
       'NOV_1990', 'DEC_1990', 'JAN_1991', 'FEB_1991', 'MAR_1991',
       'APR_1991', 'MAY_1991', 'JUN_1991', 'JUL_1991', 'AUG_1991',
       'SEP_1991', 'OCT_1991', 'NOV_1991', 'DEC_1991', 'JAN_1992',
       'FEB_1992', 'MAR_1992', 'APR_1992', 'MAY_1992', 'JUN_1992',
       'JUL_1992', 'AUG_1992', 'SEP_1992', 'OCT_1992', 'NOV_1992',
       'DEC_1992', 'JAN_1993', 'FEB_1993', 'MAR_1993', 'APR_1993',
       'MAY_1993', 'JUN_1993', 'JUL_1993', 'AUG_1993', 'SEP_1993',
       'OCT_1993', 'NOV_1993', 'DEC_1993', 'JAN_1994', 'FEB_1994',
       'MAR_1994', 'APR_1994', 'MAY_1994', 'JUN_1994', 'JUL_1994',
       'AUG_1994', 'SEP_1994', 'OCT_1994', 'NOV_1994', 'DEC_1994',
       'JAN_1995', 'FEB_1995', 'MAR_1995', 'APR_1995', 'MAY_1995',
       'JUN_1995', 'JUL_1995', 'AUG_1995', 'SEP_1995', 'OCT_1995',
       'NOV_1995', 'DEC_1995', 'JAN_1996', 'FEB_1996', 'MAR_1996',
       'APR_1996', 'MAY_1996', 'JUN_1996', 'JUL_1996', 'AUG_1996',
       'SEP_1996', 'OCT_1996', 'NOV_1996', 'DEC_1996', 'JAN_1997',
       'FEB_1997', 'MAR_1997', 'APR_1997', 'MAY_1997', 'JUN_1997',
       'JUL_1997', 'AUG_1997', 'SEP_1997', 'OCT_1997', 'NOV_1997',
       'DEC_1997', 'JAN_1998', 'FEB_1998', 'MAR_1998', 'APR_1998',
       'MAY_1998', 'JUN_1998', 'JUL_1998', 'AUG_1998', 'SEP_1998',
       'OCT_1998', 'NOV_1998', 'DEC_1998', 'JAN_1999', 'FEB_1999',
       'MAR_1999', 'APR_1999', 'MAY_1999', 'JUN_1999', 'JUL_1999',
       'AUG_1999', 'SEP_1999', 'OCT_1999', 'NOV_1999', 'DEC_1999',
       'JAN_2000', 'FEB_2000', 'MAR_2000', 'APR_2000', 'MAY_2000',
       'JUN_2000', 'JUL_2000', 'AUG_2000', 'SEP_2000', 'OCT_2000',
       'NOV_2000', 'DEC_2000', 'JAN_2001', 'FEB_2001', 'MAR_2001',
       'APR_2001', 'MAY_2001', 'JUN_2001', 'JUL_2001', 'AUG_2001',
       'SEP_2001', 'OCT_2001', 'NOV_2001', 'DEC_2001', 'JAN_2002',
       'FEB_2002', 'MAR_2002', 'APR_2002', 'MAY_2002', 'JUN_2002',
       'JUL_2002', 'AUG_2002', 'SEP_2002', 'OCT_2002', 'NOV_2002',
       'DEC_2002', 'JAN_2003', 'FEB_2003', 'MAR_2003', 'APR_2003',
       'MAY_2003', 'JUN_2003', 'JUL_2003', 'AUG_2003', 'SEP_2003',
       'OCT_2003', 'NOV_2003', 'DEC_2003', 'JAN_2004', 'FEB_2004',
       'MAR_2004', 'APR_2004', 'MAY_2004', 'JUN_2004', 'JUL_2004',
       'AUG_2004', 'SEP_2004', 'OCT_2004', 'NOV_2004', 'DEC_2004',
       'JAN_2005', 'FEB_2005', 'MAR_2005', 'APR_2005', 'MAY_2005',
       'JUN_2005', 'JUL_2005', 'AUG_2005', 'SEP_2005', 'OCT_2005',
       'NOV_2005', 'DEC_2005', 'JAN_2006', 'FEB_2006', 'MAR_2006',
       'APR_2006', 'MAY_2006', 'JUN_2006', 'JUL_2006', 'AUG_2006',
       'SEP_2006', 'OCT_2006', 'NOV_2006', 'DEC_2006', 'JAN_2007',
       'FEB_2007', 'MAR_2007', 'APR_2007', 'MAY_2007', 'JUN_2007',
       'JUL_2007', 'AUG_2007', 'SEP_2007', 'OCT_2007', 'NOV_2007',
       'DEC_2007', 'JAN_2008', 'FEB_2008', 'MAR_2008', 'APR_2008',
       'MAY_2008', 'JUN_2008', 'JUL_2008', 'AUG_2008', 'SEP_2008',
       'OCT_2008', 'NOV_2008', 'DEC_2008', 'JAN_2009', 'FEB_2009',
       'MAR_2009', 'APR_2009', 'MAY_2009', 'JUN_2009', 'JUL_2009',
       'AUG_2009', 'SEP_2009', 'OCT_2009', 'NOV_2009', 'DEC_2009',
       'JAN_2010', 'FEB_2010', 'MAR_2010', 'APR_2010', 'MAY_2010',
       'JUN_2010', 'JUL_2010', 'AUG_2010', 'SEP_2010', 'OCT_2010',
       'NOV_2010', 'DEC_2010', 'JAN_2011', 'FEB_2011', 'MAR_2011',
       'APR_2011', 'MAY_2011', 'JUN_2011', 'JUL_2011', 'AUG_2011',
       'SEP_2011', 'OCT_2011', 'NOV_2011', 'DEC_2011', 'JAN_2012',
       'FEB_2012', 'MAR_2012', 'APR_2012', 'MAY_2012', 'JUN_2012',
       'JUL_2012', 'AUG_2012', 'SEP_2012', 'OCT_2012', 'NOV_2012',
       'DEC_2012', 'JAN_2013', 'FEB_2013', 'MAR_2013', 'APR_2013',
       'MAY_2013', 'JUN_2013', 'JUL_2013', 'AUG_2013', 'SEP_2013',
       'OCT_2013', 'NOV_2013', 'DEC_2013', 'JAN_2014', 'FEB_2014',
       'MAR_2014', 'APR_2014', 'MAY_2014', 'JUN_2014', 'JUL_2014',
       'AUG_2014', 'SEP_2014', 'OCT_2014', 'NOV_2014', 'DEC_2014',
       'JAN_2015', 'FEB_2015', 'MAR_2015', 'APR_2015', 'MAY_2015',
       'JUN_2015', 'JUL_2015', 'AUG_2015', 'SEP_2015', 'OCT_2015',
       'NOV_2015', 'DEC_2015', 'JAN_2016', 'FEB_2016', 'MAR_2016',
       'APR_2016', 'MAY_2016', 'JUN_2016', 'JUL_2016', 'AUG_2016',
       'SEP_2016', 'OCT_2016', 'NOV_2016', 'DEC_2016', 'JAN_2017',
       'FEB_2017', 'MAR_2017', 'APR_2017', 'MAY_2017', 'JUN_2017',
       'JUL_2017', 'AUG_2017', 'SEP_2017', 'OCT_2017', 'NOV_2017',
       'DEC_2017', 'JAN_2018', 'FEB_2018', 'MAR_2018', 'APR_2018',
       'MAY_2018', 'JUN_2018', 'JUL_2018', 'AUG_2018', 'SEP_2018',
       'OCT_2018', 'NOV_2018', 'DEC_2018', 'JAN_2019', 'FEB_2019',
       'MAR_2019', 'APR_2019', 'MAY_2019', 'JUN_2019', 'JUL_2019',
       'AUG_2019', 'SEP_2019', 'OCT_2019', 'NOV_2019', 'DEC_2019',
       'JAN_2020', 'FEB_2020', 'MAR_2020', 'APR_2020', 'MAY_2020',
       'JUN_2020', 'JUL_2020', 'AUG_2020', 'SEP_2020', 'OCT_2020',
       'NOV_2020', 'DEC_2020', 'JAN_2021', 'FEB_2021', 'MAR_2021',
       'APR_2021', 'MAY_2021', 'JUN_2021', 'JUL_2021', 'AUG_2021',
       'SEP_2021', 'OCT_2021', 'NOV_2021', 'DEC_2021', 'JAN_2022',
       'FEB_2022', 'MAR_2022', 'APR_2022', 'MAY_2022', 'JUN_2022',
       'JUL_2022', 'AUG_2022', 'SEP_2022', 'OCT_2022', 'NOV_2022',
       'DEC_2022']]

In [18]:
GwData_all['well_no'] = range(1, len(GwData_all) + 1)

GwData_all.rename(columns = {'Well_Name':'WellName'}, inplace = True)

GwData_all.rename(columns = {'PRIN_AQUIF':'Aquifer'}, inplace = True)
GwData_all.rename(columns = {'AquiferTop':'AquiferTop'}, inplace = True)
GwData_all.rename(columns = {'AquiferBot':'AquiferBot'}, inplace = True)
GwData_all.rename(columns = {'WELL_DEPTH':'Well_Depth'}, inplace = True)
GwData_all.rename(columns = {'GCP_DEM':'DEM'}, inplace = True)

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\1252940690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GwData_all['well_no'] = range(1, len(GwData_all) + 1)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\1252940690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GwData_all.rename(columns = {'Well_Name':'WellName'}, inplace = True)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\1252940690.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [19]:
#GwData_all.columns.values

In [20]:
GwData_all = GwData_all[['well_no', 'WellName', 'Aquifer', 'X_Albers', 'Y_Albers', 'SOURCE_CD',
       'STATE_CD', 'COUNTY_CD', 'USE_CD', 'State', 'County', 'Aquifer_Co',
       'AquiferTop', 'AquiferBot', 'Well_DIAM', 'Well_Depth', 'DEM',
       'Top_Scr_1', 'Bot_Scr_1', 'Top_Scr_2', 'Bot_Scr_2', 'Top_Scr_3',
       'Bot_Scr_3', 'Top_Scr_4', 'Bot_Scr_4', 'Top_Scr_5', 'Bot_Scr_5',
       'Top_Scr_6', 'Bot_Scr_6', 'JAN_1985', 'FEB_1985', 'MAR_1985',
       'APR_1985', 'MAY_1985', 'JUN_1985', 'JUL_1985', 'AUG_1985',
       'SEP_1985', 'OCT_1985', 'NOV_1985', 'DEC_1985', 'JAN_1986',
       'FEB_1986', 'MAR_1986', 'APR_1986', 'MAY_1986', 'JUN_1986',
       'JUL_1986', 'AUG_1986', 'SEP_1986', 'OCT_1986', 'NOV_1986',
       'DEC_1986', 'JAN_1987', 'FEB_1987', 'MAR_1987', 'APR_1987',
       'MAY_1987', 'JUN_1987', 'JUL_1987', 'AUG_1987', 'SEP_1987',
       'OCT_1987', 'NOV_1987', 'DEC_1987', 'JAN_1988', 'FEB_1988',
       'MAR_1988', 'APR_1988', 'MAY_1988', 'JUN_1988', 'JUL_1988',
       'AUG_1988', 'SEP_1988', 'OCT_1988', 'NOV_1988', 'DEC_1988',
       'JAN_1989', 'FEB_1989', 'MAR_1989', 'APR_1989', 'MAY_1989',
       'JUN_1989', 'JUL_1989', 'AUG_1989', 'SEP_1989', 'OCT_1989',
       'NOV_1989', 'DEC_1989', 'JAN_1990', 'FEB_1990', 'MAR_1990',
       'APR_1990', 'MAY_1990', 'JUN_1990', 'JUL_1990', 'AUG_1990',
       'SEP_1990', 'OCT_1990', 'NOV_1990', 'DEC_1990', 'JAN_1991',
       'FEB_1991', 'MAR_1991', 'APR_1991', 'MAY_1991', 'JUN_1991',
       'JUL_1991', 'AUG_1991', 'SEP_1991', 'OCT_1991', 'NOV_1991',
       'DEC_1991', 'JAN_1992', 'FEB_1992', 'MAR_1992', 'APR_1992',
       'MAY_1992', 'JUN_1992', 'JUL_1992', 'AUG_1992', 'SEP_1992',
       'OCT_1992', 'NOV_1992', 'DEC_1992', 'JAN_1993', 'FEB_1993',
       'MAR_1993', 'APR_1993', 'MAY_1993', 'JUN_1993', 'JUL_1993',
       'AUG_1993', 'SEP_1993', 'OCT_1993', 'NOV_1993', 'DEC_1993',
       'JAN_1994', 'FEB_1994', 'MAR_1994', 'APR_1994', 'MAY_1994',
       'JUN_1994', 'JUL_1994', 'AUG_1994', 'SEP_1994', 'OCT_1994',
       'NOV_1994', 'DEC_1994', 'JAN_1995', 'FEB_1995', 'MAR_1995',
       'APR_1995', 'MAY_1995', 'JUN_1995', 'JUL_1995', 'AUG_1995',
       'SEP_1995', 'OCT_1995', 'NOV_1995', 'DEC_1995', 'JAN_1996',
       'FEB_1996', 'MAR_1996', 'APR_1996', 'MAY_1996', 'JUN_1996',
       'JUL_1996', 'AUG_1996', 'SEP_1996', 'OCT_1996', 'NOV_1996',
       'DEC_1996', 'JAN_1997', 'FEB_1997', 'MAR_1997', 'APR_1997',
       'MAY_1997', 'JUN_1997', 'JUL_1997', 'AUG_1997', 'SEP_1997',
       'OCT_1997', 'NOV_1997', 'DEC_1997', 'JAN_1998', 'FEB_1998',
       'MAR_1998', 'APR_1998', 'MAY_1998', 'JUN_1998', 'JUL_1998',
       'AUG_1998', 'SEP_1998', 'OCT_1998', 'NOV_1998', 'DEC_1998',
       'JAN_1999', 'FEB_1999', 'MAR_1999', 'APR_1999', 'MAY_1999',
       'JUN_1999', 'JUL_1999', 'AUG_1999', 'SEP_1999', 'OCT_1999',
       'NOV_1999', 'DEC_1999', 'JAN_2000', 'FEB_2000', 'MAR_2000',
       'APR_2000', 'MAY_2000', 'JUN_2000', 'JUL_2000', 'AUG_2000',
       'SEP_2000', 'OCT_2000', 'NOV_2000', 'DEC_2000', 'JAN_2001',
       'FEB_2001', 'MAR_2001', 'APR_2001', 'MAY_2001', 'JUN_2001',
       'JUL_2001', 'AUG_2001', 'SEP_2001', 'OCT_2001', 'NOV_2001',
       'DEC_2001', 'JAN_2002', 'FEB_2002', 'MAR_2002', 'APR_2002',
       'MAY_2002', 'JUN_2002', 'JUL_2002', 'AUG_2002', 'SEP_2002',
       'OCT_2002', 'NOV_2002', 'DEC_2002', 'JAN_2003', 'FEB_2003',
       'MAR_2003', 'APR_2003', 'MAY_2003', 'JUN_2003', 'JUL_2003',
       'AUG_2003', 'SEP_2003', 'OCT_2003', 'NOV_2003', 'DEC_2003',
       'JAN_2004', 'FEB_2004', 'MAR_2004', 'APR_2004', 'MAY_2004',
       'JUN_2004', 'JUL_2004', 'AUG_2004', 'SEP_2004', 'OCT_2004',
       'NOV_2004', 'DEC_2004', 'JAN_2005', 'FEB_2005', 'MAR_2005',
       'APR_2005', 'MAY_2005', 'JUN_2005', 'JUL_2005', 'AUG_2005',
       'SEP_2005', 'OCT_2005', 'NOV_2005', 'DEC_2005', 'JAN_2006',
       'FEB_2006', 'MAR_2006', 'APR_2006', 'MAY_2006', 'JUN_2006',
       'JUL_2006', 'AUG_2006', 'SEP_2006', 'OCT_2006', 'NOV_2006',
       'DEC_2006', 'JAN_2007', 'FEB_2007', 'MAR_2007', 'APR_2007',
       'MAY_2007', 'JUN_2007', 'JUL_2007', 'AUG_2007', 'SEP_2007',
       'OCT_2007', 'NOV_2007', 'DEC_2007', 'JAN_2008', 'FEB_2008',
       'MAR_2008', 'APR_2008', 'MAY_2008', 'JUN_2008', 'JUL_2008',
       'AUG_2008', 'SEP_2008', 'OCT_2008', 'NOV_2008', 'DEC_2008',
       'JAN_2009', 'FEB_2009', 'MAR_2009', 'APR_2009', 'MAY_2009',
       'JUN_2009', 'JUL_2009', 'AUG_2009', 'SEP_2009', 'OCT_2009',
       'NOV_2009', 'DEC_2009', 'JAN_2010', 'FEB_2010', 'MAR_2010',
       'APR_2010', 'MAY_2010', 'JUN_2010', 'JUL_2010', 'AUG_2010',
       'SEP_2010', 'OCT_2010', 'NOV_2010', 'DEC_2010', 'JAN_2011',
       'FEB_2011', 'MAR_2011', 'APR_2011', 'MAY_2011', 'JUN_2011',
       'JUL_2011', 'AUG_2011', 'SEP_2011', 'OCT_2011', 'NOV_2011',
       'DEC_2011', 'JAN_2012', 'FEB_2012', 'MAR_2012', 'APR_2012',
       'MAY_2012', 'JUN_2012', 'JUL_2012', 'AUG_2012', 'SEP_2012',
       'OCT_2012', 'NOV_2012', 'DEC_2012', 'JAN_2013', 'FEB_2013',
       'MAR_2013', 'APR_2013', 'MAY_2013', 'JUN_2013', 'JUL_2013',
       'AUG_2013', 'SEP_2013', 'OCT_2013', 'NOV_2013', 'DEC_2013',
       'JAN_2014', 'FEB_2014', 'MAR_2014', 'APR_2014', 'MAY_2014',
       'JUN_2014', 'JUL_2014', 'AUG_2014', 'SEP_2014', 'OCT_2014',
       'NOV_2014', 'DEC_2014', 'JAN_2015', 'FEB_2015', 'MAR_2015',
       'APR_2015', 'MAY_2015', 'JUN_2015', 'JUL_2015', 'AUG_2015',
       'SEP_2015', 'OCT_2015', 'NOV_2015', 'DEC_2015', 'JAN_2016',
       'FEB_2016', 'MAR_2016', 'APR_2016', 'MAY_2016', 'JUN_2016',
       'JUL_2016', 'AUG_2016', 'SEP_2016', 'OCT_2016', 'NOV_2016',
       'DEC_2016', 'JAN_2017', 'FEB_2017', 'MAR_2017', 'APR_2017',
       'MAY_2017', 'JUN_2017', 'JUL_2017', 'AUG_2017', 'SEP_2017',
       'OCT_2017', 'NOV_2017', 'DEC_2017', 'JAN_2018', 'FEB_2018',
       'MAR_2018', 'APR_2018', 'MAY_2018', 'JUN_2018', 'JUL_2018',
       'AUG_2018', 'SEP_2018', 'OCT_2018', 'NOV_2018', 'DEC_2018',
       'JAN_2019', 'FEB_2019', 'MAR_2019', 'APR_2019', 'MAY_2019',
       'JUN_2019', 'JUL_2019', 'AUG_2019', 'SEP_2019', 'OCT_2019',
       'NOV_2019', 'DEC_2019', 'JAN_2020', 'FEB_2020', 'MAR_2020',
       'APR_2020', 'MAY_2020', 'JUN_2020', 'JUL_2020', 'AUG_2020',
       'SEP_2020', 'OCT_2020', 'NOV_2020', 'DEC_2020', 'JAN_2021',
       'FEB_2021', 'MAR_2021', 'APR_2021', 'MAY_2021', 'JUN_2021',
       'JUL_2021', 'AUG_2021', 'SEP_2021', 'OCT_2021', 'NOV_2021',
       'DEC_2021', 'JAN_2022', 'FEB_2022', 'MAR_2022', 'APR_2022',
       'MAY_2022', 'JUN_2022', 'JUL_2022', 'AUG_2022', 'SEP_2022',
       'OCT_2022', 'NOV_2022', 'DEC_2022']]

In [21]:
# what is the source of water??
GwData_all['SOURCE_CD'].unique()

array(['GW'], dtype=object)

In [22]:
GwData_all['Aquifer'] = GwData_all['Aquifer'].fillna("NoInfo")

# Remove spaces from the 'WellName' column
GwData_all['Aquifer'] = GwData_all['Aquifer'].apply(lambda x: x.replace(" ", ""))

# Convert strings in the 'Aquifer' column to uppercase
GwData_all['Aquifer'] = GwData_all['Aquifer'].apply(lambda x: x.upper())

In [23]:
GwData_all['Aquifer'].unique()

array(['110ALVM', 'SPARTA', 'FORTPILLOW', 'SOUTHEASTLOWLANDS', 'MCNAIRY',
       'SOUTHEASTLOWLANDS/WILCOX/MCNAIRY', 'NOINFO', 'WILCOX/MCNAIRY',
       'WILCOX', 'SL/WIL', 'FORTPILLOWAQUIFER', '--', 'OZARK',
       'SOUTHEASTLOWLANDS/OZARKAQUIFER', 'UNKNOWN', 'MCNAIRY/OZ'],
      dtype=object)

In [24]:
# use only those wells that has source_CD as "GW"
GwData_all_gw = GwData_all[GwData_all['SOURCE_CD'] == 'GW']

# Drop rows with no screen info
GwData_all_f = GwData_all_gw[GwData_all_gw['Top_Scr_1'] != 0]

# Reset the index
GwData_all_f = GwData_all_f.reset_index(drop=True)

In [25]:
# Save the wells with no screen/ well depth information for future evaluation
GwData_all_noScreen = GwData_all[GwData_all['Top_Scr_1'] == 0]
GwData_all_noScreen.to_csv(f'No_ScreenInfoWells_Rgn_{rgn}.csv')

In [26]:
len(GwData_all_gw), len(GwData_all_f), len(GwData_all_gw)- len(GwData_all_f)

(6967, 6808, 159)

In [27]:
GwData_all_f['well_no'] = range(1, len(GwData_all_f) + 1)
for i in range(len(GwData_all_f)):
    iName = str(GwData_all_f.iloc[i,1])
    iUseCd = str(GwData_all_f.iloc[i,8])
#    print(iName, iUseCd)
    GwData_all_f.iloc[i,1] = str(iName+iUseCd)

In [28]:
GwData_all.iloc[:,1:21].head()

,WellName,Aquifer,X_Albers,Y_Albers,SOURCE_CD,STATE_CD,COUNTY_CD,USE_CD,State,County,Aquifer_Co,AquiferTop,AquiferBot,Well_DIAM,Well_Depth,DEM,Top_Scr_1,Bot_Scr_1,Top_Scr_2,Bot_Scr_2
0,350230090152601z,110ALVM,519455,1346785,GW,5,35.0,AG,Arkansas,0,0,0.000000,0.00000,12.00,110.0,64.285820,90.000000,110.000000,0.0,0.0
1,350231090160601z,110ALVM,518455,1346785,GW,5,35.0,AG,Arkansas,0,0,0.000000,0.00000,12.00,110.0,63.634903,90.000000,110.000000,0.0,0.0
2,35000090000001z,110ALVM,542455,1343785,GW,5,35.0,AG,Arkansas,0,0,0.000000,0.00000,16.00,110.0,103.458221,90.000000,110.000000,0.0,0.0
3,35000090000002z,110ALVM,542455,1343785,GW,5,35.0,AG,Arkansas,0,0,0.000000,0.00000,16.00,110.0,103.458221,90.000000,110.000000,0.0,0.0
4,TN0000450_PALMERz,SPARTA,541455,1344785,GW,47,0.0,0,Tennessee,0,0,199.457524,995.19035,7.99,0.0,90.651520,204.457524,229.457524,0.0,0.0


# PackageData

In [36]:
dfPackageData = GwData_all_f[['well_no', 'WellName', 'Aquifer', 'X_Albers', 'Y_Albers', 'Well_DIAM', 'Top_Scr_1', 'Bot_Scr_1', 'DEM']]

# DEM (Land surface elevation) is in meters, screens depth is in feet. 

dfPackageData['Top_Scr_1'] = dfPackageData['Top_Scr_1']* float(0.3048)
dfPackageData['Bot_Scr_1'] = dfPackageData['Bot_Scr_1']* float(0.3048)

dfPackageData['ScrnTop'] = dfPackageData['DEM'] - dfPackageData['Top_Scr_1']
dfPackageData['ScrnBot'] = dfPackageData['DEM'] - dfPackageData['Bot_Scr_1']

dfPackageData['Rw:radius'] = dfPackageData['Well_DIAM']* float(0.0254)
dfPackageData['bottom'] = float()
dfPackageData['strt'] = float()
dfPackageData['condeqn'] = str('THEIM')

dfPackageData['ngwfnodes'] = int()

dfPackageData = dfPackageData[['well_no', 'Rw:radius', 'bottom', 'strt', 'condeqn',
       'ngwfnodes', 'WellName', 'Aquifer', 'X_Albers', 'Y_Albers', 'ScrnTop', 'ScrnBot']]

dfPackageData['well_no'] = dfPackageData['well_no'].astype(int)

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\74277965.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPackageData['Top_Scr_1'] = dfPackageData['Top_Scr_1']* float(0.3048)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\74277965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPackageData['Bot_Scr_1'] = dfPackageData['Bot_Scr_1']* float(0.3048)
C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\74277965.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [37]:
dfPackageData.loc[dfPackageData['ScrnBot']>dfPackageData['ScrnTop']]

,well_no,Rw:radius,bottom,strt,condeqn,ngwfnodes,WellName,Aquifer,X_Albers,Y_Albers,ScrnTop,ScrnBot


In [38]:
dfPackageData['Aquifer'] = dfPackageData['Aquifer'].fillna("NoInfo")

# Remove spaces from the 'WellName' column
dfPackageData['Aquifer'] = dfPackageData['Aquifer'].apply(lambda x: x.replace(" ", ""))

# Convert strings in the 'Aquifer' column to uppercase
dfPackageData['Aquifer'] = dfPackageData['Aquifer'].apply(lambda x: x.upper())

In [39]:
dfPackageData['Aquifer'].unique()

array(['110ALVM', 'SPARTA', 'FORTPILLOW', 'SOUTHEASTLOWLANDS', 'MCNAIRY',
       'SOUTHEASTLOWLANDS/WILCOX/MCNAIRY', 'NOINFO', 'WILCOX/MCNAIRY',
       'WILCOX', 'SL/WIL', 'FORTPILLOWAQUIFER', '--', 'OZARK',
       'SOUTHEASTLOWLANDS/OZARKAQUIFER', 'UNKNOWN', 'MCNAIRY/OZ'],
      dtype=object)

# Connection Data

### Find the list of sand cells that are within the range of screen top and screen bottom

In [48]:
def find_Sand_cells_for_well_screens(aquifer_df, scrnTop, scrnBot):
    
    sandCells_for_screen = []
    screen_top = scrnTop
    screen_bot = scrnBot

    for index, row in aquifer_df.iterrows():
        cell_id = row['cellid']
        cell_top = row['cellTop']
        cell_bot = row['cellBot']
        
        # find sand cells
        if cell_top > screen_bot and cell_bot <= screen_top:
            sandCells_for_screen.append(int(cell_id))
    
    return sandCells_for_screen

In [40]:
dfPackageData.head()

,well_no,Rw:radius,bottom,strt,condeqn,ngwfnodes,WellName,Aquifer,X_Albers,Y_Albers,ScrnTop,ScrnBot
0,1,0.304800,0.0,0.0,THEIM,0,350230090152601zAG,110ALVM,519455,1346785,36.853820,30.757820
1,2,0.304800,0.0,0.0,THEIM,0,350231090160601zAG,110ALVM,518455,1346785,36.202903,30.106903
2,3,0.406400,0.0,0.0,THEIM,0,35000090000001zAG,110ALVM,542455,1343785,76.026221,69.930221
3,4,0.406400,0.0,0.0,THEIM,0,35000090000002zAG,110ALVM,542455,1343785,76.026221,69.930221
4,5,0.202946,0.0,0.0,THEIM,0,TN0000450_PALMERz0,SPARTA,541455,1344785,28.332867,20.712867


In [34]:
RH_basic.head()

,cellid,cellTop,cellBot,IC,K,K33,SS,SY,X,Y,IND,MidPoint
0,1,103.59500,102.09500,102.126034,50.81019,5.081019,0.997474,0.3,587455,1591785,31,102.84500
1,2,102.09500,87.97634,102.126034,50.81019,5.081019,0.997474,0.3,587455,1591785,0,95.03567
2,3,100.64600,99.14600,101.870874,50.81019,5.081019,0.997474,0.3,588455,1591785,2,99.89600
3,4,99.14600,87.97634,101.870874,50.81019,5.081019,0.997474,0.3,588455,1591785,2,93.56117
4,5,100.76554,99.26554,101.917743,50.81019,5.081019,0.997474,0.3,589455,1591785,2,100.01554


In [49]:
connectionsForWells = {}


for index, row in dfPackageData.iterrows():
    screen_top = row['ScrnTop']
    screen_bottom = row['ScrnBot']
    
    rX = row['X_Albers']
    rY = row['Y_Albers']
    rAq = row['Aquifer']
    rWellNo = row['WellName']
    
    # get a temporary dataframe for this specific x,y location
    temp_df = RH_basic.loc[(RH_basic['X'] == int(rX)) & (RH_basic['Y'] == int(rY))]
    
    # filter only sand cells
    
    sands_df = temp_df.loc[temp_df['IND'] == int(1)]
    
    # Now we actually only need cellId, cellTop, and cellBot
    
    aquifer_df = sands_df[['cellid', 'cellTop', 'cellBot']]

    
    # Get the corresponding sand cells
    sandCells = find_Sand_cells_for_well_screens(aquifer_df, scrnTop= screen_top, scrnBot = screen_bottom)
     
    if not len(sandCells)==0:
        connectionsForWells[int(row['well_no'])] = sandCells

In [50]:
connectionsForWells

{5: [308845],
 6: [307182],
 7: [306545],
 8: [305658],
 11: [303981],
 12: [304726],
 13: [305357],
 14: [302956],
 15: [302914],
 16: [303478],
 17: [302586],
 18: [300043, 300045],
 19: [300757],
 20: [298956],
 21: [298872],
 22: [296960],
 23: [295424],
 24: [293758],
 26: [293486],
 27: [292601],
 28: [292484],
 29: [291756],
 30: [289383],
 35: [282931],
 36: [279386, 279388],
 39: [274570, 274572],
 40: [275335],
 41: [274539],
 44: [268868],
 45: [263676],
 46: [252489, 252491],
 47: [251640],
 48: [304014],
 50: [301412],
 51: [301296],
 53: [285215],
 54: [282552],
 56: [251123],
 94: [193806],
 215: [192315],
 273: [186700],
 571: [246550],
 572: [246550],
 573: [240938],
 574: [240938],
 575: [239808],
 576: [233150],
 578: [225413],
 579: [224432],
 580: [225020],
 581: [222423],
 584: [212155],
 585: [193009],
 589: [174401],
 591: [168722],
 592: [244140],
 595: [240336],
 597: [234348],
 599: [220638],
 600: [215675],
 601: [213560],
 602: [208400],
 603: [208322],
 60

### Write Connection data

In [51]:
connectionColumns=['well_no', 'icon', 'Dummy_ID', 'scrn_top', 'scrn_bot', 'hk_skin', 'radius_skin']
dfConnection=pd.DataFrame(columns=connectionColumns)

rh_well_Nos=list(dfPackageData["well_no"])

for well in rh_well_Nos:
    intWell = int(well)
    if intWell in connectionsForWells:
        wellConnections =[]
        twellConnections = connectionsForWells[intWell]
        
        for iFinal in twellConnections:
            intiFinal = int(iFinal)
            if not intiFinal in wellConnections:
                wellConnections.append(intiFinal)

        # set total number of connections
        total_ngwfnodes = len(wellConnections)
        #print(total_ngwfnodes)

        # Update the ngwfnodes (number of connections) for each well of package data

        dfPackageData["ngwfnodes"].loc[dfPackageData["well_no"] == intWell] = total_ngwfnodes


        # update connection data
        for rh_connection in range(0, total_ngwfnodes):
            rh_connection_no = rh_connection+1
            rh_connections_cellNo = int(wellConnections[rh_connection])
            #print(well, rh_connection_no, rh_connections_cellNo)

            # values to append
            values = [well, rh_connection_no, rh_connections_cellNo, 0, 0, 0, 0]

            dfConnection.loc[len(dfConnection)] = values

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\987200256.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfPackageData["ngwfnodes"].loc[dfPackageData["well_no"] == intWell] = total_ngwfnodes
C:\Users\rhasan1\AppData\Local\Temp\ipyke

C:\Users\rhasan1\AppData\Local\Temp\ipykernel_8664\987200256.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfPackageData["ngwfnodes"].loc[dfPackageData["well_no"] == intWell] = total_ngwfnodes
C:\Users\rhasan1\AppData\Local\Temp\ipyke

In [52]:
dfPackageData.head()

,well_no,Rw:radius,bottom,strt,condeqn,ngwfnodes,WellName,Aquifer,X_Albers,Y_Albers,ScrnTop,ScrnBot
0,1,0.304800,0.0,0.0,THEIM,0,350230090152601zAG,110ALVM,519455,1346785,36.853820,30.757820
1,2,0.304800,0.0,0.0,THEIM,0,350231090160601zAG,110ALVM,518455,1346785,36.202903,30.106903
2,3,0.406400,0.0,0.0,THEIM,0,35000090000001zAG,110ALVM,542455,1343785,76.026221,69.930221
3,4,0.406400,0.0,0.0,THEIM,0,35000090000002zAG,110ALVM,542455,1343785,76.026221,69.930221
4,5,0.202946,0.0,0.0,THEIM,1,TN0000450_PALMERz0,SPARTA,541455,1344785,28.332867,20.712867


In [55]:
dfConnection.head()

,well_no,icon,Dummy_ID,scrn_top,scrn_bot,hk_skin,radius_skin
0,5,1,308845,0,0,0,0
1,6,1,307182,0,0,0,0
2,7,1,306545,0,0,0,0
3,8,1,305658,0,0,0,0
4,11,1,303981,0,0,0,0


### Remove wells with 0 connections

In [54]:
zeroConnectionWells = list(dfPackageData['well_no'].loc[dfPackageData['ngwfnodes'] == int(0)])
dfPackageData = dfPackageData[~dfPackageData['well_no'].isin(zeroConnectionWells)]

# Stress Period

In [56]:
GwData_all_f.head()

,well_no,WellName,Aquifer,X_Albers,Y_Albers,SOURCE_CD,STATE_CD,COUNTY_CD,USE_CD,State,...,MAR_2022,APR_2022,MAY_2022,JUN_2022,JUL_2022,AUG_2022,SEP_2022,OCT_2022,NOV_2022,DEC_2022
0,1,350230090152601zAG,110ALVM,519455,1346785,GW,5,35.0,AG,Arkansas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,350231090160601zAG,110ALVM,518455,1346785,GW,5,35.0,AG,Arkansas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,35000090000001zAG,110ALVM,542455,1343785,GW,5,35.0,AG,Arkansas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,35000090000002zAG,110ALVM,542455,1343785,GW,5,35.0,AG,Arkansas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,TN0000450_PALMERz0,SPARTA,541455,1344785,GW,47,0.0,0,Tennessee,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
stressPeriodData = GwData_all_f[['well_no', 'JAN_2004', 'FEB_2004', 'MAR_2004',
       'APR_2004', 'MAY_2004', 'JUN_2004', 'JUL_2004', 'AUG_2004',
       'SEP_2004', 'OCT_2004', 'NOV_2004', 'DEC_2004', 'JAN_2005',
       'FEB_2005', 'MAR_2005', 'APR_2005', 'MAY_2005', 'JUN_2005',
       'JUL_2005', 'AUG_2005', 'SEP_2005', 'OCT_2005', 'NOV_2005',
       'DEC_2005', 'JAN_2006', 'FEB_2006', 'MAR_2006', 'APR_2006',
       'MAY_2006', 'JUN_2006', 'JUL_2006', 'AUG_2006', 'SEP_2006',
       'OCT_2006', 'NOV_2006', 'DEC_2006', 'JAN_2007', 'FEB_2007',
       'MAR_2007', 'APR_2007', 'MAY_2007', 'JUN_2007', 'JUL_2007',
       'AUG_2007', 'SEP_2007', 'OCT_2007', 'NOV_2007', 'DEC_2007',
       'JAN_2008', 'FEB_2008', 'MAR_2008', 'APR_2008', 'MAY_2008',
       'JUN_2008', 'JUL_2008', 'AUG_2008', 'SEP_2008', 'OCT_2008',
       'NOV_2008', 'DEC_2008', 'JAN_2009', 'FEB_2009', 'MAR_2009',
       'APR_2009', 'MAY_2009', 'JUN_2009', 'JUL_2009', 'AUG_2009',
       'SEP_2009', 'OCT_2009', 'NOV_2009', 'DEC_2009', 'JAN_2010',
       'FEB_2010', 'MAR_2010', 'APR_2010', 'MAY_2010', 'JUN_2010',
       'JUL_2010', 'AUG_2010', 'SEP_2010', 'OCT_2010', 'NOV_2010',
       'DEC_2010', 'JAN_2011', 'FEB_2011', 'MAR_2011', 'APR_2011',
       'MAY_2011', 'JUN_2011', 'JUL_2011', 'AUG_2011', 'SEP_2011',
       'OCT_2011', 'NOV_2011', 'DEC_2011', 'JAN_2012', 'FEB_2012',
       'MAR_2012', 'APR_2012', 'MAY_2012', 'JUN_2012', 'JUL_2012',
       'AUG_2012', 'SEP_2012', 'OCT_2012', 'NOV_2012', 'DEC_2012',
       'JAN_2013', 'FEB_2013', 'MAR_2013', 'APR_2013', 'MAY_2013',
       'JUN_2013', 'JUL_2013', 'AUG_2013', 'SEP_2013', 'OCT_2013',
       'NOV_2013', 'DEC_2013', 'JAN_2014', 'FEB_2014', 'MAR_2014',
       'APR_2014', 'MAY_2014', 'JUN_2014', 'JUL_2014', 'AUG_2014',
       'SEP_2014', 'OCT_2014', 'NOV_2014', 'DEC_2014', 'JAN_2015',
       'FEB_2015', 'MAR_2015', 'APR_2015', 'MAY_2015', 'JUN_2015',
       'JUL_2015', 'AUG_2015', 'SEP_2015', 'OCT_2015', 'NOV_2015',
       'DEC_2015', 'JAN_2016', 'FEB_2016', 'MAR_2016', 'APR_2016',
       'MAY_2016', 'JUN_2016', 'JUL_2016', 'AUG_2016', 'SEP_2016',
       'OCT_2016', 'NOV_2016', 'DEC_2016', 'JAN_2017', 'FEB_2017',
       'MAR_2017', 'APR_2017', 'MAY_2017', 'JUN_2017', 'JUL_2017',
       'AUG_2017', 'SEP_2017', 'OCT_2017', 'NOV_2017', 'DEC_2017',
       'JAN_2018', 'FEB_2018', 'MAR_2018', 'APR_2018', 'MAY_2018',
       'JUN_2018', 'JUL_2018', 'AUG_2018', 'SEP_2018', 'OCT_2018',
       'NOV_2018', 'DEC_2018', 'JAN_2019', 'FEB_2019', 'MAR_2019',
       'APR_2019', 'MAY_2019', 'JUN_2019', 'JUL_2019', 'AUG_2019',
       'SEP_2019', 'OCT_2019', 'NOV_2019', 'DEC_2019', 'JAN_2020',
       'FEB_2020', 'MAR_2020', 'APR_2020', 'MAY_2020', 'JUN_2020',
       'JUL_2020', 'AUG_2020', 'SEP_2020', 'OCT_2020', 'NOV_2020',
       'DEC_2020']]

In [58]:
stressPeriodData.head()

,well_no,JAN_2004,FEB_2004,MAR_2004,APR_2004,MAY_2004,JUN_2004,JUL_2004,AUG_2004,SEP_2004,...,MAR_2020,APR_2020,MAY_2020,JUN_2020,JUL_2020,AUG_2020,SEP_2020,OCT_2020,NOV_2020,DEC_2020
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.349512e+07,1.390406e+07,1.349512e+07,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.349512e+07,1.390406e+07,1.349512e+07,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,10753083.0,11078934.0,10753083.0,0.0,0.0,...,0.0,0.0,0.0,7.624913e+06,7.624913e+06,7.624913e+06,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,45184672.0,45184672.0,45184672.0,45184672.0,0.0,...,0.0,0.0,0.0,3.522721e+07,3.524350e+07,3.522721e+07,0.0,0.0,0.0,0.0
4,5,153450000.0,148190000.0,159340000.0,156600000.0,164300000.0,153600000.0,162130000.0,159340000.0,152100000.0,...,115010000.0,84600000.0,109120000.0,1.308000e+08,1.233800e+08,1.159400e+08,108600000.0,106640000.0,81600000.0,91140000.0


In [59]:
stressPeriodData = stressPeriodData[~stressPeriodData['well_no'].isin(zeroConnectionWells)]

In [60]:
(stressPeriodData < 0).any().any()

False

In [62]:
# Convert from GAL/month to m3/day
stressPeriodData.iloc[:, 1:] = stressPeriodData.iloc[:, 1:] * (-1) * (0.000124452)
stressPeriodData.iloc[:, 1:] = stressPeriodData.iloc[:, 1:].round(4)

In [63]:
monthList = ['JAN_2004', 'FEB_2004', 'MAR_2004',
       'APR_2004', 'MAY_2004', 'JUN_2004', 'JUL_2004', 'AUG_2004',
       'SEP_2004', 'OCT_2004', 'NOV_2004', 'DEC_2004', 'JAN_2005',
       'FEB_2005', 'MAR_2005', 'APR_2005', 'MAY_2005', 'JUN_2005',
       'JUL_2005', 'AUG_2005', 'SEP_2005', 'OCT_2005', 'NOV_2005',
       'DEC_2005', 'JAN_2006', 'FEB_2006', 'MAR_2006', 'APR_2006',
       'MAY_2006', 'JUN_2006', 'JUL_2006', 'AUG_2006', 'SEP_2006',
       'OCT_2006', 'NOV_2006', 'DEC_2006', 'JAN_2007', 'FEB_2007',
       'MAR_2007', 'APR_2007', 'MAY_2007', 'JUN_2007', 'JUL_2007',
       'AUG_2007', 'SEP_2007', 'OCT_2007', 'NOV_2007', 'DEC_2007',
       'JAN_2008', 'FEB_2008', 'MAR_2008', 'APR_2008', 'MAY_2008',
       'JUN_2008', 'JUL_2008', 'AUG_2008', 'SEP_2008', 'OCT_2008',
       'NOV_2008', 'DEC_2008', 'JAN_2009', 'FEB_2009', 'MAR_2009',
       'APR_2009', 'MAY_2009', 'JUN_2009', 'JUL_2009', 'AUG_2009',
       'SEP_2009', 'OCT_2009', 'NOV_2009', 'DEC_2009', 'JAN_2010',
       'FEB_2010', 'MAR_2010', 'APR_2010', 'MAY_2010', 'JUN_2010',
       'JUL_2010', 'AUG_2010', 'SEP_2010', 'OCT_2010', 'NOV_2010',
       'DEC_2010', 'JAN_2011', 'FEB_2011', 'MAR_2011', 'APR_2011',
       'MAY_2011', 'JUN_2011', 'JUL_2011', 'AUG_2011', 'SEP_2011',
       'OCT_2011', 'NOV_2011', 'DEC_2011', 'JAN_2012', 'FEB_2012',
       'MAR_2012', 'APR_2012', 'MAY_2012', 'JUN_2012', 'JUL_2012',
       'AUG_2012', 'SEP_2012', 'OCT_2012', 'NOV_2012', 'DEC_2012',
       'JAN_2013', 'FEB_2013', 'MAR_2013', 'APR_2013', 'MAY_2013',
       'JUN_2013', 'JUL_2013', 'AUG_2013', 'SEP_2013', 'OCT_2013',
       'NOV_2013', 'DEC_2013', 'JAN_2014', 'FEB_2014', 'MAR_2014',
       'APR_2014', 'MAY_2014', 'JUN_2014', 'JUL_2014', 'AUG_2014',
       'SEP_2014', 'OCT_2014', 'NOV_2014', 'DEC_2014', 'JAN_2015',
       'FEB_2015', 'MAR_2015', 'APR_2015', 'MAY_2015', 'JUN_2015',
       'JUL_2015', 'AUG_2015', 'SEP_2015', 'OCT_2015', 'NOV_2015',
       'DEC_2015', 'JAN_2016', 'FEB_2016', 'MAR_2016', 'APR_2016',
       'MAY_2016', 'JUN_2016', 'JUL_2016', 'AUG_2016', 'SEP_2016',
       'OCT_2016', 'NOV_2016', 'DEC_2016', 'JAN_2017', 'FEB_2017',
       'MAR_2017', 'APR_2017', 'MAY_2017', 'JUN_2017', 'JUL_2017',
       'AUG_2017', 'SEP_2017', 'OCT_2017', 'NOV_2017', 'DEC_2017',
       'JAN_2018', 'FEB_2018', 'MAR_2018', 'APR_2018', 'MAY_2018',
       'JUN_2018', 'JUL_2018', 'AUG_2018', 'SEP_2018', 'OCT_2018',
       'NOV_2018', 'DEC_2018', 'JAN_2019', 'FEB_2019', 'MAR_2019',
       'APR_2019', 'MAY_2019', 'JUN_2019', 'JUL_2019', 'AUG_2019',
       'SEP_2019', 'OCT_2019', 'NOV_2019', 'DEC_2019', 'JAN_2020',
       'FEB_2020', 'MAR_2020', 'APR_2020', 'MAY_2020', 'JUN_2020',
       'JUL_2020', 'AUG_2020', 'SEP_2020', 'OCT_2020', 'NOV_2020',
       'DEC_2020']

In [64]:
month_index = {}
for month in monthList:
    month_index[month] = monthList.index(month) +1

In [65]:
month_index

{'JAN_2004': 1,
 'FEB_2004': 2,
 'MAR_2004': 3,
 'APR_2004': 4,
 'MAY_2004': 5,
 'JUN_2004': 6,
 'JUL_2004': 7,
 'AUG_2004': 8,
 'SEP_2004': 9,
 'OCT_2004': 10,
 'NOV_2004': 11,
 'DEC_2004': 12,
 'JAN_2005': 13,
 'FEB_2005': 14,
 'MAR_2005': 15,
 'APR_2005': 16,
 'MAY_2005': 17,
 'JUN_2005': 18,
 'JUL_2005': 19,
 'AUG_2005': 20,
 'SEP_2005': 21,
 'OCT_2005': 22,
 'NOV_2005': 23,
 'DEC_2005': 24,
 'JAN_2006': 25,
 'FEB_2006': 26,
 'MAR_2006': 27,
 'APR_2006': 28,
 'MAY_2006': 29,
 'JUN_2006': 30,
 'JUL_2006': 31,
 'AUG_2006': 32,
 'SEP_2006': 33,
 'OCT_2006': 34,
 'NOV_2006': 35,
 'DEC_2006': 36,
 'JAN_2007': 37,
 'FEB_2007': 38,
 'MAR_2007': 39,
 'APR_2007': 40,
 'MAY_2007': 41,
 'JUN_2007': 42,
 'JUL_2007': 43,
 'AUG_2007': 44,
 'SEP_2007': 45,
 'OCT_2007': 46,
 'NOV_2007': 47,
 'DEC_2007': 48,
 'JAN_2008': 49,
 'FEB_2008': 50,
 'MAR_2008': 51,
 'APR_2008': 52,
 'MAY_2008': 53,
 'JUN_2008': 54,
 'JUL_2008': 55,
 'AUG_2008': 56,
 'SEP_2008': 57,
 'OCT_2008': 58,
 'NOV_2008': 59,
 'DEC_

In [68]:
# make lists of tuples for stress values-- [(period, wellNo, pumping rate),] for easy processing
df_melted = stressPeriodData.melt(id_vars=['well_no'], var_name='month', value_name='value')

df_melted['month'] = df_melted['month'].map(month_index)

stressValues = list(df_melted[['month', 'well_no', 'value']].itertuples(index=False, name=None))

In [69]:
stressPeriodData

,well_no,JAN_2004,FEB_2004,MAR_2004,APR_2004,MAY_2004,JUN_2004,JUL_2004,AUG_2004,SEP_2004,...,MAR_2020,APR_2020,MAY_2020,JUN_2020,JUL_2020,AUG_2020,SEP_2020,OCT_2020,NOV_2020,DEC_2020
4,5,-19097.1594,-18442.5419,-19830.1817,-19489.1832,-20447.4636,-19115.8272,-20177.4028,-19830.1817,-18929.1492,...,-14313.2245,-10528.6392,-13580.2022,-16278.3216,-15354.8878,-14428.9649,-13515.4872,-13271.5613,-10155.2832,-11342.5553
5,6,-75964.2563,-68212.1412,-72646.3660,-72468.3996,-82137.0755,-77546.0412,-80169.4894,-78587.7044,-74633.8644,...,-59413.3848,-56414.0916,-56867.0969,-65635.9848,-72337.7250,-74189.5708,-67801.4496,-69405.6359,-61491.7332,-64583.1209
6,7,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,...,-593.4714,-582.5391,-484.9854,-580.6681,-572.8726,-603.6645,-558.6318,-460.5687,-476.2820,-573.9606
7,8,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,...,-4734.5274,-4228.6301,-4357.9357,-3846.4380,-5412.0441,-6106.3618,-5389.3939,-6223.2223,-5954.7793,-5094.1937
10,11,-16415.8411,-15414.5003,-17125.7153,-19093.4258,-24756.8630,-24660.1638,-26315.4999,-30370.2705,-29517.5254,...,-16990.6848,-17936.0222,-23969.8286,-29905.8156,-38532.6665,-34598.6516,-29644.4664,-24398.0679,-18626.7308,-16666.6118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6803,6804,-1480.3743,-1384.8663,-1480.3743,-1432.6203,-1480.3743,-1432.6203,-1480.3743,-1480.3743,-1432.6203,...,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988
6804,6805,-1480.3743,-1384.8663,-1480.3743,-1432.6203,-1480.3743,-1432.6203,-1480.3743,-1480.3743,-1432.6203,...,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988
6805,6806,-1480.3743,-1384.8663,-1480.3743,-1432.6203,-1480.3743,-1432.6203,-1480.3743,-1480.3743,-1432.6203,...,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988
6806,6807,-1480.3743,-1384.8663,-1480.3743,-1432.6203,-1480.3743,-1432.6203,-1480.3743,-1480.3743,-1432.6203,...,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988,-1443.9988,-1397.4182,-1443.9988,-1397.4182,-1443.9988


In [70]:
stressValues

[(1, 5, -19097.1594),
 (1, 6, -75964.2563),
 (1, 7, -0.0),
 (1, 8, -0.0),
 (1, 11, -16415.8411),
 (1, 12, -60686.5288),
 (1, 13, -0.0),
 (1, 14, -0.0),
 (1, 15, -0.0),
 (1, 16, -0.0),
 (1, 17, -0.0),
 (1, 18, -0.0),
 (1, 19, -76581.5382),
 (1, 20, -49652.6144),
 (1, 21, -0.0),
 (1, 22, -0.0),
 (1, 23, -0.0),
 (1, 24, -85995.0875),
 (1, 26, -0.0),
 (1, 27, -0.0),
 (1, 28, -0.0),
 (1, 29, -0.0),
 (1, 30, -0.0),
 (1, 35, -0.0),
 (1, 36, -0.0),
 (1, 39, -1041.6632),
 (1, 40, -2696.7504),
 (1, 41, -0.0),
 (1, 44, -0.0),
 (1, 45, -0.0),
 (1, 46, -0.0),
 (1, 47, -0.0),
 (1, 48, -0.0),
 (1, 50, -69.4442),
 (1, 51, -0.0),
 (1, 53, -2244.2056),
 (1, 54, -2249.221),
 (1, 56, -0.0),
 (1, 94, -0.0),
 (1, 215, -0.0),
 (1, 273, -0.0),
 (1, 571, -5983.7766),
 (1, 572, -416.6653),
 (1, 573, -0.0),
 (1, 574, -0.0),
 (1, 575, -0.0),
 (1, 576, -104.1663),
 (1, 578, -1064.8113),
 (1, 579, -4278.5353),
 (1, 580, -0.0),
 (1, 581, -185.1846),
 (1, 584, -0.0),
 (1, 585, -0.0),
 (1, 589, -3865.728),
 (1, 591, -

In [71]:
# Creating a DataFrame
full_period_df = pd.DataFrame(stressValues, columns=['Period', 'Well_No', 'Rate'])

In [72]:
full_period_df.head()

,Period,Well_No,Rate
0,1,5,-19097.1594
1,1,6,-75964.2563
2,1,7,-0.0000
3,1,8,-0.0000
4,1,11,-16415.8411


In [73]:
# convert from m3/d to MGD

# Calculate statistics
mean_h = full_period_df['Rate'].mean()
max_h = full_period_df['Rate'].max()
min_h = full_period_df['Rate'].min()
percentiles_h = np.percentile(full_period_df['Rate'], [25, 50, 75])
mode_h = full_period_df['Rate'].mode()[0]

# Print statistics
print(f"Mean of Head: {mean_h*(0.00021996)} MGD")
print(f"Max of Head: {max_h*(0.00021996)} MGD")
print(f"Min of Head: {min_h*(0.00021996)} MGD")
print(f"25th percentile of Head: {percentiles_h[0]}")
print(f"50th percentile (median) of Head: {percentiles_h[1]}")
print(f"75th percentile of Head: {percentiles_h[2]}")
print(f"Mode of Head: {mode_h}")

Mean of Head: -0.554589155449223 MGD
Max of Head: -0.0 MGD
Min of Head: -28.063477117728 MGD
25th percentile of Head: -623.5045
50th percentile (median) of Head: -0.0
75th percentile of Head: 0.0
Mode of Head: -0.0


In [74]:
# Dictionary to organizing data by periods
# format -- {'Period' : 'a dataframe that has all the wells and their pumping rate for this period'}
period_data_dict = {}

for period, well, rate in stressValues:
    period_key = f'Period{period}'
    if period_key not in period_data_dict:
        period_data_dict[period_key] = []
    period_data_dict[period_key].append([well, 'rate', rate])

# Converting lists to DataFrame for each period
for period_key in period_data_dict:
    df = pd.DataFrame(period_data_dict[period_key], columns=[0, 1, 2])
    period_data_dict[period_key] = df

In [75]:
period_data_dict

{'Period1':         0     1           2
 0       5  rate -19097.1594
 1       6  rate -75964.2563
 2       7  rate     -0.0000
 3       8  rate     -0.0000
 4      11  rate -16415.8411
 ..    ...   ...         ...
 196  6804  rate  -1480.3743
 197  6805  rate  -1480.3743
 198  6806  rate  -1480.3743
 199  6807  rate  -1480.3743
 200  6808  rate   -259.7728
 
 [201 rows x 3 columns],
 'Period2':         0     1           2
 0       5  rate -18442.5419
 1       6  rate -68212.1412
 2       7  rate     -0.0000
 3       8  rate     -0.0000
 4      11  rate -15414.5003
 ..    ...   ...         ...
 196  6804  rate  -1384.8663
 197  6805  rate  -1384.8663
 198  6806  rate  -1384.8663
 199  6807  rate  -1384.8663
 200  6808  rate   -243.0133
 
 [201 rows x 3 columns],
 'Period3':         0     1           2
 0       5  rate -19830.1817
 1       6  rate -72646.3660
 2       7  rate     -0.0000
 3       8  rate     -0.0000
 4      11  rate -17125.7153
 ..    ...   ...         ...
 196  6804  ra

# Rename well_no -- as we lost some wells due to zero connections

In [80]:
packageDataRh_Final = dfPackageData[['well_no', 'Rw:radius', 'bottom', 'strt', 'condeqn', 'ngwfnodes', 
                                     'WellName', 'Aquifer', 'X_Albers', 'Y_Albers']]

In [81]:
old_new_well_no_dict = {} # {old: new}
for i in range(len(packageDataRh_Final)):
    new_well_no = int(i+1)
    original_well_no = dfPackageData.iloc[i,0]
    
    #print(new_well_no, original_well_no)
    
    # update the dataframe and the dict
    packageDataRh_Final.iloc[i,0] = new_well_no
    old_new_well_no_dict[original_well_no] = new_well_no

In [82]:
connectionDataRh_Final = dfConnection[['well_no', 'icon', 'Dummy_ID', 'scrn_top', 'scrn_bot', 'hk_skin',
       'radius_skin']]

In [83]:
# update connection block
for i in range(len(connectionDataRh_Final)):
    old_well_no = dfConnection.iloc[i,0]
    new_well_no = old_new_well_no_dict[old_well_no]
    #print(old_well_no, new_well_no)
    # update the well_no
    connectionDataRh_Final.iloc[i,0] = new_well_no

In [84]:
# update period block
connection_in_rh = list(packageDataRh_Final["well_no"])
data=period_data_dict
period_data = {}
for period, df in data.items():
    
    
    df[0] = df[0].astype(int)
    df[2] = df[2].astype(float)
    #print(df)
    
    df_temp = df[[0, 1, 2]]
    
    # update the well_no-- as we are remaining the well no
    for i in range(len(df_temp)):
        old_well_no = df_temp.iloc[i,0]
        new_well_no = old_new_well_no_dict[old_well_no]
        #print(old_well_no, new_well_no)
        
        # update the well_no
        df_temp.iloc[i,0] = new_well_no
    #print(df_temp)
    period_data[period] = df_temp

### Update package data bottom, strt, and radius column

In [88]:
packageDataRh = packageDataRh_Final[['well_no', 'Rw:radius', 'bottom', 'strt', 'condeqn', 'ngwfnodes',
       'WellName', 'Aquifer', 'X_Albers', 'Y_Albers']]

connectionDataRh = connectionDataRh_Final[['well_no', 'icon', 'Dummy_ID', 'scrn_top', 'scrn_bot', 'hk_skin',
       'radius_skin']]

In [89]:
packageDataRh.head()

,well_no,Rw:radius,bottom,strt,condeqn,ngwfnodes,WellName,Aquifer,X_Albers,Y_Albers
4,1,0.202946,0.0,0.0,THEIM,1,TN0000450_PALMERz0,SPARTA,541455,1344785
5,2,0.202946,0.0,0.0,THEIM,1,TN0000450_DAVISz0,SPARTA,532455,1346785
6,3,0.202946,0.0,0.0,THEIM,1,198z0,SPARTA,553455,1347785
7,4,0.202946,0.0,0.0,THEIM,1,206z0,SPARTA,552455,1348785
10,5,0.202946,0.0,0.0,THEIM,1,TN0000126z0,SPARTA,572455,1350785


In [90]:
for i in range(len(packageDataRh)):
    if packageDataRh.iloc[i,1] == float(0):
        packageDataRh.iloc[i,1] = float(0.24384) # replacing with dummy value

In [91]:
# Update well bottom for the package data
for i in range(len(packageDataRh)):
    
    wellNo = int(packageDataRh.iloc[i,0])
 
    # get corresponding 3D cells-related to connestion block 
    rhCellsList = list(connectionDataRh["Dummy_ID"].loc[connectionDataRh["well_no"]==wellNo])
    
    # initiate a best bottom
    bestBtm = float(list(RH_basic["cellBot"].loc[RH_basic["cellid"] == int(rhCellsList[0])])[0])
    
    for rhCell in rhCellsList: 
        tempBtm = float(list(RH_basic["cellBot"].loc[RH_basic["cellid"] == int(rhCell)])[0])
        
        # update best botm
        if tempBtm < bestBtm:
            bestBtm = tempBtm
    
    # update well botm 
    packageDataRh.iloc[i,2] = round(float(bestBtm),4)

In [92]:
# Update well strt value for the package data-- elevation of the top most cell for the time being due to lack of info

for i in range(len(packageDataRh)):
    iWel = int(packageDataRh.iloc[i,0])
    
    # get corresponding 3D cells 
    rCellsFirst = list(connectionDataRh["Dummy_ID"].loc[connectionDataRh["well_no"]==iWel])[0]
    
    ix = list(RH_basic["X"].loc[RH_basic["cellid"] == rCellsFirst])[0]
    iy = list(RH_basic["Y"].loc[RH_basic["cellid"] == rCellsFirst])[0]
    topElv = RH_basic["cellTop"].loc[(RH_basic["X"] == ix) & (RH_basic["Y"] == iy)].max()
    
    packageDataRh.iloc[i,3] = round(float(topElv),4)  

# Lets set scrnTop and scrnBot in the connection block as 0-- for the time being
for i in range(len(connectionDataRh)):
    connectionDataRh.iloc[i,3] = float(0)
    connectionDataRh.iloc[i,4] = float(0)

### Package Data With CellID for reference

In [93]:
RH_basic

,cellid,cellTop,cellBot,IC,K,K33,SS,SY,X,Y,IND,MidPoint
0,1,103.59500,102.09500,102.126034,50.810190,5.081019,0.997474,0.3,587455,1591785,31,102.845000
1,2,102.09500,87.97634,102.126034,50.810190,5.081019,0.997474,0.3,587455,1591785,0,95.035670
2,3,100.64600,99.14600,101.870874,50.810190,5.081019,0.997474,0.3,588455,1591785,2,99.896000
3,4,99.14600,87.97634,101.870874,50.810190,5.081019,0.997474,0.3,588455,1591785,2,93.561170
4,5,100.76554,99.26554,101.917743,50.810190,5.081019,0.997474,0.3,589455,1591785,2,100.015540
...,...,...,...,...,...,...,...,...,...,...,...,...
309808,309809,-177.17000,-179.87000,70.769905,23.741698,2.374170,0.000100,0.1,534455,1342785,1,-178.520000
309809,309810,-179.87000,-230.53999,70.769905,23.740855,2.373587,0.000100,0.1,534455,1342785,0,-205.204995
309810,309811,-230.53999,-305.63303,70.765731,0.021946,0.300000,0.000100,0.1,534455,1342785,12,-268.086510
309811,309812,-305.63303,-383.81879,70.760398,6.748272,0.674827,0.000100,0.1,534455,1342785,13,-344.725910


In [94]:
# add corresponding 3d cells and thir sand/shale indicator to the package data block for future reference 
packageDataRh_CellID = packageDataRh[packageDataRh.columns.values]
packageDataRh_CellID['CellId'] = str()
packageDataRh_CellID['IndList'] = str() 

for iW in range(len(packageDataRh_CellID['well_no'])):
    iWel = int(packageDataRh_CellID.iloc[iW,0])
    
    # get corresponding 3D cells 
    rCells = list(connectionDataRh["Dummy_ID"].loc[connectionDataRh["well_no"]==iWel])
    
    # Ind
    indList = []
    for r in rCells:
        iInd = list(RH_basic["IND"].loc[RH_basic["cellid"] == r])[0]
        indList.append(iInd)
        
    packageDataRh_CellID.iloc[iW,-2] = str(rCells)
    packageDataRh_CellID.iloc[iW,-1] = str(indList)

# Write package

In [95]:
packageFilename = toSavePath + "/" +regionName + "PackageData.csv"
periodFilename = toSavePath + "/" +regionName + "PeriodData.csv"
packageDataRh_CellID.to_csv(packageFilename)
full_period_df.to_csv(periodFilename)

In [99]:
packageDataRh = packageDataRh[['well_no', 'Rw:radius', 'bottom', 'strt', 'condeqn', 'ngwfnodes', 'WellName']]

packagedata_df = packageDataRh[packageDataRh.columns.values]
connectiondata_df = connectionDataRh[connectionDataRh.columns.values]

In [100]:
# Calculate the nmawwells value 
nmawwells = len(packagedata_df)

# Formatting PACKAGEDATA and CONNECTIONDATA
formatted_packagedata = '\n'.join(packagedata_df.apply(lambda x: '\t'.join(x.astype(str)), axis=1))
formatted_connectiondata = '\n'.join(connectiondata_df.apply(lambda x: '\t'.join(x.astype(str)), axis=1))

# Template for the MAW package
template = f"""# MAW: Multi-Aquifer Well package file created on 2/10/2024 by Rafin Hasan
BEGIN OPTIONS
#   PRINT_INPUT
    SAVE_FLOWS
END OPTIONS

BEGIN DIMENSIONS 
    nmawwells {nmawwells}
END DIMENSIONS
 
BEGIN PACKAGEDATA
#well_no\tRw:radius\tbottom\tstrt\tcondeqn\tngwfnodes\tWellName
{formatted_packagedata}
END PACKAGEDATA

BEGIN CONNECTIONDATA
#well_no\ticon\tDummy_ID\tscrn_top\tscrn_bot\thk_skin\tradius_skin
{formatted_connectiondata}
END CONNECTIONDATA
"""

# Formatting PERIOD Block
for period, df in period_data.items():
    period_values = '\n'.join(df.iloc[:, 0].astype(str) + ' ' + df.iloc[:, 1].astype(str) + ' ' + df.iloc[:, -1].astype(str))
    template += f"\nBEGIN PERIOD {period.replace('Period', '').strip()}\n{period_values}\nEND PERIOD\n"

# filename
filename = toSavePath + "/" +regionName + ".maw6"

# Write the content
with open(filename, 'w') as file:
    file.write(template)

print(f"File '{filename}' has been created.")

File 'C:\Users\rhasan1\Documents\Summer-24\0.NewMsThesis\MERAS_GeneratePackages_New\0.MakeMaw\new_packahes_path/Region4.maw6' has been created.
